In [2]:
import re
import pandas as pd
import pickle
import os

In [3]:
articles_metadata = pd.read_json("../data/article_metadata_with_filepaths.json")
articles_metadata

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes,filepath,pop_id
0,2011,Bos et al.,Political Communication,How the Media Shape Perceptions of Right-Wing ...,article,y,y,y,None,Political Communication/2011 - Bos et al. - Ho...,0
1,2019,Blassnig et al.,Political Communication,Hitting a Nerve: Populist News Articles Lead t...,article,y,y,y,None,Political Communication/2019 - Blassing et al....,1
2,2017,Caramani,American Political Science Review,Will vs. Reason: The Populist and Technocratic...,article,y,y,None,None,American Political Science Review/2017 - Caram...,2
3,2020,"Wuttke,Schimpf, Schoen",American Political Science Review,When the Whole Is Greater than the Sum of Its ...,article,y,y,None,None,American Political Science Review/2020 - Wuttk...,3
4,2014,Treib,Journal of European Public Policy,"The voter says no, but nobody listens: causes ...",article,y,y,None,None,Journal of European Public Policy/2014 - Treib...,4
...,...,...,...,...,...,...,...,...,...,...,...
537,2018,Grzymala-Busse,Current History,Poland’s Path to Illiberalism,article,y,y,None,None,Current History/2018 - Grzymala-Busse - Poland...,537
538,2017,Eichengreen,Current History,The Populist Turn in American Politics,article,y,y,None,None,Current History/2017 - Eichengreen - The Popul...,538
539,2014,Mudde,Current History,The Far Right and the European Elections,article,y,y,None,None,Current History/2014 - Mudde - The Far Right a...,539
540,2020,Cano,European History Quarterly,Printing the People,article,y,y,None,None,European History Quarterly/2020 - Cano - Print...,540


In [12]:
articles_metadata[articles_metadata["pop_id"]==17]

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes,filepath,pop_id
17,2018,Rooduijn,European Political Science Review,What unites the voter bases of populist partie...,article,y,y,None,None,European Political Science Review/2018 - Roodu...,17


In [48]:
articles_metadata = articles_metadata[articles_metadata["filepath"].apply(bool)].copy()

In [49]:
articles_sample = articles_metadata.sample(10, random_state=1)
articles_sample

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes,filepath,pop_id
524,2016,Brewer,Forum: A Journal of Applied Research in Contem...,Populism in American Politics,article,y,y,None,None,Forum A Journal of Applied Research in Contemp...,524
142,2016,"Bobba, McDonnell",South European Society and Politics,Different Types of Right-Wing Populist Discour...,article,y,y,None,None,South European Society and Politics/2016 - Bob...,142
65,2019,Shah,Journal of Democracy,Pakistan: Voting Under Military Tutelage,article,y,y,y,None,Journal of Democracy/2019 - Shah - Pakistan_Vo...,65
41,2017,Applebaum,Journal of Democracy,Britain After Brexit: A Transformed Political ...,article,y,y,y,None,Journal of Democracy/2017 - Applebaum - Britai...,41
268,2016,Aslanidis,Political Studies,Is Populism an Ideology? A Refutation and a Ne...,article,y,y,None,None,Political Studies/2016 - Aslanidis - Is Populi...,268
504,2019,Appel,Critical Review,Can the EU Stop Eastern Europe's Illiberal Turn?,article,y,y,None,None,Critical Review/2019 - Appel - Can the EU Sto...,504
225,2015,"Kettell, Kerr",British Politics,Editorial,editorial,y,y,y,nevim je to jen editorial kde je jednou slovo ...,"British Politics/2015 - Kettell, Kerr - Editor...",225
189,2019,Csehi,Democratization,"Neither episodic, nor destined to failure? The...",article,y,y,None,None,Democratization/2019 - Csehi - Neither episodi...,189
47,2017,"Puddington, Roylance",Journal of Democracy,The Freedom House Survey for 2016: The Dual Th...,article,y,y,y,None,"Journal of Democracy/2017 - Puddington, Roylan...",47
0,2011,Bos et al.,Political Communication,How the Media Shape Perceptions of Right-Wing ...,article,y,y,y,None,Political Communication/2011 - Bos et al. - Ho...,0


In [50]:
sample_pop_ids = [62, 377, 188, 364, 499, 228, 376, 90, 227, 330]

In [51]:
# create folder for filtered texts
!mkdir ../data/large_data/articles_filteredtexts

mkdir: ../data/large_data/articles_filteredtexts: File exists


In [52]:
path = "../data/large_data/articles_filteredtexts/"

In [53]:
filteredtext = "In the middle of 2011, both Turkey and Thailand held national elections,124 While Turkish premier Recep Tayyip Erdo¢gan claimed his third popular mandate, his self-exiled Thai doppelgänger Thaksin Shinawatra—whose parties have now won five general elections since 2001—saw his younger sister and stand-in Yingluck Shinawatra assume the premiership."

In [54]:
re.sub("([\.|\:|\?\,])(\d+)", r"\1", filteredtext)

'In the middle of 2011, both Turkey and Thailand held national elections, While Turkish premier Recep Tayyip Erdo¢gan claimed his third popular mandate, his self-exiled Thai doppelgänger Thaksin Shinawatra—whose parties have now won five general elections since 2001—saw his younger sister and stand-in Yingluck Shinawatra assume the premiership.'

In [55]:
spliters = ["References\n",
            "Bibliography\n",
            "Notes\n",
            "Acknowledgments\n"]
def text_cleaning(filteredtext):
    filteredtext_len = len(filteredtext)
    usable_spliters = []
    for spliter in spliters + [spliter.upper() for spliter in spliters]:
        if spliter in filteredtext:
            keeping_len = len(filteredtext.partition(spliter)[0])
            if keeping_len > filteredtext_len / 2:
                usable_spliters.append((keeping_len, spliter))
    if bool(usable_spliters):
        keeping_index = min(usable_spliters)[0]
        filteredtext = filteredtext[:keeping_index]
    filteredtext = filteredtext.replace("-\n", "")
    filteredtext = re.sub("([\.|\:|\?\,])(\d+)", r"\1", filteredtext)
    return filteredtext

In [71]:
def get_filteredtext(pop_id, pct=2, tofile=True):
    filename = "textblocks_pop_id_{}.pickle".format(str(pop_id))
    pages_back = pickle.load(open("../data/large_data/articles_textblocks/" + filename, "rb"))
    rawtext = ""
    for p in pages_back:
        for tb in p:
            rawtext += tb[4]
    rawtext_per_page = len(rawtext) / len(pages_back)
    filteredtext = ""
    end_here = False
    for p in pages_back:
        accepted_tbs = []
        last_included = False
        for tb in p:
            tb_text = tb[4]
            # main filtering condition:
            #if tb_text.count("\n") > 2:
            if last_included == False:
                if len(tb_text) >= rawtext_per_page * pct / 100:
                    accepted_tbs.append(tb_text)
                    last_included = True
            else:
                accepted_tbs.append(tb_text)
            if tb_text in spliters + [spliter.upper() for spliter in spliters]:
                end_here = True
                break
        accepted_tbs_reversed = []
        last_included = False
        for tb_text in reversed(accepted_tbs):
            if last_included == False:
                if len(tb_text) >= rawtext_per_page * pct / 100:
                    accepted_tbs_reversed.append(tb_text)
                    last_included = True
            else:
                accepted_tbs_reversed.append(tb_text)
        accepted_tbs_reversed = [tb for tb in accepted_tbs_reversed if tb[0] != "<"]
        filteredtext += " ".join(reversed(accepted_tbs_reversed))
        if end_here == True:
            break
    filteredtext = text_cleaning(filteredtext)
    if tofile:
        newfile = "filteredtext_pop_id_{}.txt".format(str(pop_id))
        with open(path + newfile, "w") as f:
            f.write(filteredtext)
    else:
        return filteredtext

In [76]:
get_filteredtext(110,tofile=False)

'Escaping Populism – Safeguarding Minority Rights:\nNon-majoritarian Dynamics in European Policy-making*\n EIKO THIELEMANN and NATASCHA ZAUN\nLondon School of Economics and Political Science\n Abstract\nContrary to earlier predictions, communitarization in the area of asylum policies has not led to an\nerosion of refugee rights. Instead, there is growing evidence that EU asylum harmonization has\nsafeguarded existing standards and even enhanced the rights of asylum-seekers and refugees in\nEurope. We seek to explain this by building on the insights of principal-agent theory. We argue\nthat delegation to supranational institutions can strengthen non-majoritarian policy dynamics and\nshield EU policy-makers from populist pressures for further immigration restrictions that national\ngovernments are often confronted with. We support this argument empirically through a systematic longitudinal analysis of the evolution of EU asylum policies. In particular, we seek to assess\nthe motivation f

In [73]:
filteredtext_len = len(filteredtext)
usable_spliters = []
for spliter in spliters + [spliter.upper() for spliter in spliters]:
    if spliter in filteredtext:
        keeping_len = len(filteredtext.partition(spliter)[0])
        if keeping_len > filteredtext_len / 2:
            usable_spliters.append((keeping_len, spliter))
if bool(usable_spliters):
    keeping_index = min(usable_spliters)[0]
    filteredtext = filteredtext[:keeping_index]
filteredtext = filteredtext.replace("-\n", "")
filteredtext = re.sub("([\.|\:|\?\,])(\d+)", r"\1", filteredtext)


In [74]:
#!mkdir ../data/large_data/sample_filteredtext
for id in sample_pop_ids:
    filteredtext = get_filteredtext(id, tofile=False)
    sample_path = "../data/large_data/sample_filteredtext/"
    newfile = "filteredtext_pop_id_{}.txt".format(str(id))
    with open(sample_path + newfile, "w") as f:
        f.write(filteredtext)

In [75]:
path = "../data/large_data/articles_filteredtexts/"
for id in articles_metadata["pop_id"]:
    get_filteredtext(id)